In [1]:
from pathlib import Path
import rasterio
import numpy as np
from rasterio.warp import reproject, Resampling
import geopandas as gpd
from raster_manipulation import save_raster, clip_raster, exact_raster_reprojection, fast_raster_reprojection

In [2]:
#country_names = [f.name for f in Path("./").iterdir() if len(f.name)==3]
country_names = ['GMB']

for country in country_names:
    print('')
    print(country)
    
    # Clipping all NTL rasters according to GADM boundaries
    
    path_in = Path("./"+country)
    
    aoi_name = "gadm36_"+country+".gpkg"
    aoi_in = Path("./"+country+"/misc_data") / aoi_name
    aoi = gpd.read_file(aoi_in, layer='gadm36_'+country+'_0')
    
    for NTL_file in Path("./"+country+"/NTL").iterdir():
        clipped, aff, crs = clip_raster(NTL_file, aoi)
        save_raster(NTL_file, clipped, aff)
        print(NTL_file.name, 'clipped.')
    
    # Opening last NTL raster to get the targeted resolution
    
    ntl_in = Path("./"+country+"/NTL") / NTL_file.name
    ntl = rasterio.open(ntl_in)
    
    # Reprojecting HRSL raster

    misc_data_files = [file.name for file in Path("./"+country+"/misc_data").iterdir()]
    if ("HRSL_"+country+"_NTL-resolution.tif" not in misc_data_files):
        if len([file for file in misc_data_files if 'population_'+country.lower() in file])==1:
            hrsl_name = [file for file in misc_data_files if 'population_'+country.lower() in file][0]
            hrsl_in = Path("./"+country+"/misc_data") / hrsl_name
            hrsl = rasterio.open(hrsl_in)
            
            print('Upsampling', hrsl_name)
            hrsl_ntl_reso, _ = exact_raster_reprojection(hrsl, ntl)

            ntl_out = "HRSL_"+country+"_NTL-resolution.tif"
            path_out = Path("./"+country+"/misc_data") / ntl_out
            save_raster(path_out, hrsl_ntl_reso, ntl.transform)
            clipped, aff, crs = clip_raster(path_out, aoi)
            save_raster(path_out, clipped, aff)
            
        else:
            print('Unable to find one HRSL file.')
                
    # Reprojecting WorldPop rasters
    
    worldpop_files = [file for file in Path("./"+country+"/worldpop").iterdir() if 'ppp' in file.name]
    if len(worldpop_files) > 0:
        for file in worldpop_files:
            if "worldpop_"+file.stem[-4:]+'_'+country+"_NTL-resolution.tif" not in [file.name for file in Path("./"+country+"/worldpop").iterdir()]:
                worldpop_in = Path("./"+country+"/worldpop") / file.name
                worldpop = rasterio.open(worldpop_in)
                worldpop_rd = worldpop.read(1)

                print('Upsampling',file.name)
                #worldpop_ntl_reso, _ = exact_raster_reprojection(worldpop, ntl) # For exact but slow reprojection
                worldpop_ntl_reso = fast_raster_reprojection(worldpop, ntl)

                ntl_out = "worldpop_"+file.stem[-4:]+'_'+country+"_NTL-resolution.tif"
                path_out = Path("./"+country+"/worldpop") / ntl_out
                save_raster(path_out, worldpop_ntl_reso, ntl.transform)
                clipped, aff, crs = clip_raster(path_out, aoi)
                save_raster(path_out, clipped, aff)
    else:
        print('Unable to find a WorldPop file.')


GMB
NTL_2014_GMB.tif clipped.
NTL_2014_GMB_local-correction.tif clipped.
